In [ ]:
#Import libraries
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import PIL as pil
import os
import PIL
import pathlib
import glob

#Set to GPU
devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(devices[0], True)

#Path
images_source = pathlib.Path("D:\s4532664\COMP3710\ISIC2018_Task1-2_Training_Data")

#Input images
input_dir = images_source / "ISIC2018_Task1-2_Training_Input_x2\*.jpg"

#Segmentation images (label images)
seg_dir = images_source / "ISIC2018_Task1_Training_GroundTruth_x2\*.png"
                             
#Setup arrays for training, testing and validation datasets
X_train = None
X_test = None
X_valid = None
y_train = None
y_test = None
y_valid = None

training_data = None
validation_data = None
testing_data = None

In [ ]:
print(input_dir)
print(seg_dir)

In [ ]:
#Process every filename in the array as an image, using TF functions. 
#Normalise every image by either 0 or 1 and divide it by 255 for scaling purposes

def convert_file_image(input_file, seg_file):
    input_image = tf.cast(tf.image.convert_image_dtype(tf.image.decode_jpeg(tf.io.read_file(input_file), \
                                                                    channels=3), tf.uint8), tf.float32) / 255.0
    seg_image = tf.cast(tf.image.decode_png(tf.io.read_file(seg_file), channels=3), tf.float32) / 255.0
    
    return input_image, seg_image

In [ ]:
#Load images from disk into training, testing and validation arrays (DO DOCSTRING)
def load_data(xtrain, xtest, xvalid, ytrain, ytest, yvalid, inputdir, segdir):
    
    xtrain = sorted(glob.glob(str(inputdir)))
    ytrain = sorted(glob.glob(str(segdir)))
    
    #Split the images into 3 datasets (Training - 50%, Validation - 25%, Testing - 25%)
    
    half_length = int(len(xtrain)/2)
    quarter_length_ceil = int(tf.math.ceil(len(xtrain)/4))
    print(quarter_length_ceil)
    
    xtest = xtrain[-(quarter_length_ceil-1):]
    xvalid = xtrain[half_length:half_length + quarter_length_ceil]
    xtrain = xtrain[0:half_length]
    
    ytest = ytrain[-(quarter_length_ceil-1):]
    yvalid = ytrain[half_length:half_length + quarter_length_ceil]
    ytrain = ytrain[0:half_length]
     
    traindata = tf.data.Dataset.from_tensor_slices((xtrain, ytrain))
    validdata = tf.data.Dataset.from_tensor_slices((xvalid, yvalid))
    testdata = tf.data.Dataset.from_tensor_slices((xtest, ytest))
    
    traindata = traindata.map(convert_file_image)
    validdata = validdata.map(convert_file_image)
    testdata = testdata.map(convert_file_image)
    
    return traindata, validdata, testdata

In [ ]:
training_data, validation_data, testing_data = load_data(X_train, X_test, X_valid, y_train, \
                                                         y_test, y_valid, input_dir, seg_dir)